In [1]:
# import libraries
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

# set gpu
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# split data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# flatten the data and normalize it
x_train = x_train.reshape(-1, 28*28).astype("float32")/255.0
x_test = x_test.reshape(-1, 28*28).astype("float32")/255.0

In [32]:
# pretrained model
model = keras.models.load_model("my_model1/")
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 575,050
Trainable params: 575,050
Non-trainable

In [33]:
# Freeze the layers of model
model.trainable = False

In [34]:
base_input = model.layers[0].input
base_output = model.layers[-2].output
final_output = layers.Dense(10, name='add_layer1')(base_output)

In [35]:
new_model = tf.keras.Model(inputs=base_input, outputs=final_output)

In [36]:
new_model.compile(optimizer='Adam',
             loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics = ["accuracy"])

In [38]:
new_model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 8s - loss: 0.2412 - accuracy: 0.9524 - 8s/epoch - 4ms/step
Epoch 2/5
1875/1875 - 7s - loss: 0.0276 - accuracy: 0.9918 - 7s/epoch - 3ms/step
Epoch 3/5
1875/1875 - 7s - loss: 0.0238 - accuracy: 0.9924 - 7s/epoch - 4ms/step
Epoch 4/5
1875/1875 - 6s - loss: 0.0222 - accuracy: 0.9927 - 6s/epoch - 3ms/step
Epoch 5/5
1875/1875 - 6s - loss: 0.0212 - accuracy: 0.9930 - 6s/epoch - 3ms/step


In [40]:
# save the fine tuned model
new_model.save('add_layer1')

INFO:tensorflow:Assets written to: add_layer1\assets


In [41]:
m = keras.models.load_model('add_layer1/')

In [42]:
print(m.summary())

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 add_layer1 (Dense)          (None, 10)                650       
                                                                 
Total params: 575,050
Trainable params: 650
Non-trainable p